In [ ]:
# project_id = 'project-5400504384186300846'
# from google.colab import auth
# auth.authenticate_user()
!gcloud auth application-default login

In [2]:
import glob, os
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, precision_score, recall_score

import tensorflow as tf
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.linalg import SparseVector
from google.cloud import bigquery
from google.cloud import storage

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', 200)

In [3]:
!pyspark --version

22/01/02 17:34:22 WARN Utils: Your hostname, bangbuis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
22/01/02 17:34:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, Java HotSpot(TM) 64-Bit Server VM, 1.8.0_261
Branch HEAD
Compiled by user ubuntu on 2021-10-06T12:46:30Z
Revision 5d45a415f3a29898d92380380cfd82bfc7f579ea
Url https://github.com/apache/spark
Type --help for more information.


In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sql_context = pyspark.SQLContext(sc)

22/01/02 17:34:23 WARN Utils: Your hostname, bangbuis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
22/01/02 17:34:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/02 17:34:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/bangbui/anaconda3/envs/learn_py3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
!gsutil cp gs://momovn-analytics/trang.le4/fraud_promotion_v2/data/train201909.avro /tmp/referralFraud/train201909.avro

CommandException: No URLs matched: gs://momovn-analytics/trang.le4/fraud_promotion_v2/data/train201909.avro


In [ ]:
!bq --location=US extract --destination_format=AVRO --compression=SNAPPY \
--synchronous_mode=false \
--quiet=true \
--project_id=project-5400504384186300846 \
project-5400504384186300846:team_analytics.MERCHANT_LABEL_ALL_05_08_POPUP_V1_07 \
/tmp/merchant_detection/labels/2021/all_05_08_popup_v1_07/*.avro